In [1]:
import os.path
import pandas as pd
import numpy as np 
import env

In [2]:
def get_df():
    if os.path.isfile('zillow.csv'):
        df = get_zillow_csv()
    else:
        df = wrangle_zillow()
    return df

In [3]:
def get_zillow_sql():
     return '''
        select 
            bathroomcnt,
            bedroomcnt,
            calculatedfinishedsquarefeet,
            taxvaluedollarcnt,
            taxamount,
            fips
        from `properties_2017`
        join `predictions_2017`using(`parcelid`)
        where propertylandusetypeid = 261 and `transactiondate` >= '2017-05-01' and `transactiondate` <= '2017-06-30'
    '''

In [4]:
def get_zillow_url():
    return f'mysql+pymysql://{env.user}:{env.password}@{env.host}/zillow'

In [5]:
def wrangle_zillow():
    df = pd.read_sql(get_zillow_sql(), get_zillow_url())
    bathcount = round(df.bathroomcnt.mean())
    bedcount = round(df.bedroomcnt.mean())
    df.bathroomcnt.replace(0, bathcount, inplace=True)
    df.bedroomcnt.replace(0, bedcount, inplace=True)
    df.dropna(inplace=True)
    df['tax_rate'] = df.taxvaluedollarcnt / df.taxamount
    df.fips = df.fips.astype(int)
    
    return df

In [6]:
def get_zillow_csv():
    df = pd.read_csv('zillow.csv')
    df.drop(columns=['Unnamed: 0'], inplace=True)
    return df